In [1]:
 #accessing netcdf files from Hector's folders
import os, glob, sys
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts")
sys.path.append("//nobackup//amondal//Python//mit_geos_analysis")
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts//HandleMITgcm")
#from mds_store import openmdsdataset
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
# How about visualization?
%matplotlib inline
import netCDF4
from netCDF4 import Dataset
#from xmitgcm import open_mdsdataset
from GetNCDataSet import *
from MITllc2160Depth import *



import xgcm
from spectral_analysis_code import *

from xmitgcm import open_mdsdataset
from dask.distributed import Client
client = Client(memory_limit='75GB',n_workers = 40, threads_per_worker=1)

In [2]:
oldnpdir = "/nobackup/amondal/NCData/Hector_CCS_Kappa/"
npdir = "/nobackup/amondal/NCData/20220425_diagnostic_vars_SNAPS/"
kpphbl = np.load(npdir + 'KPPhbl_test.npy')
latc = np.load(oldnpdir + 'latc_test.npy')
lonc = np.load(oldnpdir + 'lonc_test.npy')
u = np.load(npdir + 'UVEL_test.npy')
v = np.load(npdir + 'VVEL_test.npy')
t = np.load(npdir + 'THETA_test.npy')
s = np.load(npdir + 'SALT_test.npy')

# TAKING ONLY 1 TIMESTEP


In [3]:
thk = scipy.io.loadmat('../Hector_Python_Scripts/thk90')
thickness = thk['thk90'][0,0:90]
depth = thk['dpt90'][0,0:90]
layer_min = 20
layer_max = 30

lat_list = latc[:,0]
lon_list = lonc[0]
time_list = np.arange(0,u.shape[0])
depth_list = depth[layer_min:layer_max]

In [4]:
uval = np.swapaxes(np.swapaxes(u, 0,3),1,2)
vval = np.swapaxes(np.swapaxes(v, 0,3),1,2)
tval = np.swapaxes(np.swapaxes(t, 0,3),1,2)
sval = np.swapaxes(np.swapaxes(s, 0,3),1,2)
U = xr.DataArray(uval[:, :,layer_min:layer_max, :], dims = ['lon','lat','Zlayers','time'], coords = {"lon":lon_list, "lat": lat_list, "Zlayers": depth_list, "time": time_list}, name = "U")
V = xr.DataArray(vval[:, :,layer_min:layer_max, :], dims = ['lon','lat','Zlayers','time'], coords = {"lon":lon_list, "lat": lat_list, "Zlayers": depth_list, "time": time_list}, name = "V")
T = xr.DataArray(tval[:, :,layer_min:layer_max, :], dims = ['lon','lat','Zlayers','time'], coords = {"lon":lon_list, "lat": lat_list, "Zlayers": depth_list, "time": time_list}, name = r"$\Theta$")
S = xr.DataArray(sval[:, :,layer_min:layer_max, :], dims = ['lon','lat','Zlayers','time'], coords = {"lon":lon_list, "lat": lat_list, "Zlayers": depth_list, "time": time_list}, name = "Salt")

In [24]:
thickness[20:30]

array([10.69, 11.7 , 12.76, 13.87, 15.03, 16.22, 17.45, 18.7 , 19.97,
       21.27])

In [21]:
depth_list

array([ 88.175,  99.37 , 111.6  , 124.915, 139.365, 154.99 , 171.825,
       189.9  , 209.235, 229.855])

In [8]:
depth_tile.shape

(24, 10, 786, 980)

In [7]:
U.shape

(980, 786, 10, 24)

In [14]:
depth_tile_x.shape

(980, 786, 10, 24)

In [7]:
sys.path.append("//nobackup//amondal//Python//Joes_kpp_code//12_16_21//")
import fld_tools as ft
Uval = U.values
Vval = V.values
shsq_Joe = ft.get_shsq(Uval,Vval)
print('Shear calculated')

Shear calculated


In [8]:
def ufunc_get_shsq(U, V):
    return xr.apply_ufunc(ft.get_shsq, U, V, dask='parallelized')

In [9]:
import time

In [12]:
begin = time.time()
shsq_Joe = ft.get_shsq(Uval,Vval)
end = time.time()
print(end-begin)

43.81620955467224


In [13]:
begin = time.time()
shsq_xr = ufunc_get_shsq(U, V)
end = time.time()
print(end - begin)

42.73566508293152


The above cells seem to indicate that the xarray'd version of Joe's code isn't faster than the original. However, at the two different filesizes, the two only vary slightly. I'll use the xarray version just so that I can keep using xarray type files.

Below, I'll take Joe's code and adapt them into something that I can use. 

In [10]:
# Joe's original code. He had developed an class to handle a lot of this and so we have to modify things a bit
def compute_rho_mitgcm(self):
        # get dims
        (nx,ny,nz,nt) = self.T.shape;
                
        rho0 = 1035; g = 9.81;
        p = (np.cumsum(self.thknss,axis=2)-0.5*self.thknss)*g*rho0/10000 + 10;
        SA = np.zeros([nx,ny,nz,nt]); CT = np.zeros([nx,ny,nz,nt]); self.rho = np.zeros([nx,ny,nz,nt]);
        for k in range(0,nz):
            for t in range(0,nt):
                SA[:,:,k,t] = gsw.SA_from_SP(self.S[:,:,k,t],p[:,:,k,0],np.squeeze(self.pgrid[:,:,0]),np.squeeze(self.pgrid[:,:,1]));
                CT[:,:,k,t] = gsw.CT_from_t(SA[:,:,k,t],self.T[:,:,k,t],p[:,:,k,0]);
                self.rho[:,:,k,t] = gsw.rho(SA[:,:,k,t],CT[:,:,k,t],p[:,:,k,0]);

def compute_pot_rho_down_mitgcm(self):
        # get dims
        (nx,ny,nz,nt) = self.T.shape;
                
        rho0 = 1035; g = 9.81;
        p = (np.cumsum(self.thknss,axis=2)-0.5*self.thknss)*g*rho0/10000 + 10;
        SA = np.zeros([nx,ny,nz,nt]); CT = np.zeros([nx,ny,nz,nt]); pot_rho = np.zeros([nx,ny,nz,nt]);
        for k in range(0,nz-1):
            for t in range(0,nt):
                SA[:,:,k,t] = gsw.SA_from_SP(self.S[:,:,k,t],p[:,:,k+1,0],np.squeeze(self.pgrid[:,:,0]),np.squeeze(self.pgrid[:,:,1]));
                CT[:,:,k,t] = gsw.CT_from_t(SA[:,:,k,t],self.T[:,:,k,t],p[:,:,k+1,0]);
                pot_rho[:,:,k,t] = gsw.rho(SA[:,:,k,t],CT[:,:,k,t],p[:,:,k+1,0]);

        return pot_rho;

In [15]:
T.dims

('lon', 'lat', 'Zlayers', 'time')

In [16]:
T.Zlayers

<xarray.DataArray 'Zlayers' (Zlayers: 10)>
array([ 88.175,  99.37 , 111.6  , 124.915, 139.365, 154.99 , 171.825, 189.9  ,
       209.235, 229.855])
Coordinates:
  * Zlayers  (Zlayers) float64 88.17 99.37 111.6 124.9 ... 189.9 209.2 229.9

In [ ]:
depth

In [32]:
def calculate_p_joe(thk, layer_min, layer_max):
    g = 9.81; rho0 = 1035;
    p = (np.cumsum(thk,axis=0)-0.5*thk)*g*rho0/10000 + 10;
    return p[layer_min:layer_max]
def calculate_p_joe_xarray(depthsx, layer_min, layer_max):
    g = 9.81; rho0 = 1035;
    p = depthsx*g*rho0 / 10000 + 10
    return p.rename('Pressures');

In [29]:
ptest = (np.cumsum(thickness)-0.5*thickness)
ptest[20:30]

array([ 88.175,  99.37 , 111.6  , 124.915, 139.365, 154.99 , 171.825,
       189.9  , 209.235, 229.855])

In [28]:
depth_tile_x[10,10,:,2]

<xarray.DataArray 'Depths' (Zlayers: 10)>
array([ 88.175,  99.37 , 111.6  , 124.915, 139.365, 154.99 , 171.825,
       189.9  , 209.235, 229.855])
Coordinates:
    lon      >f4 -127.5
    lat      >f4 36.04
  * Zlayers  (Zlayers) float64 88.17 99.37 111.6 124.9 ... 189.9 209.2 229.9
    time     int64 2

In [34]:
p = calculate_p_joe_xarray(depth_tile_x, layer_min, layer_max)

In [51]:
lat_mesh, lon_mesh = np.meshgrid(S.lat, S.lon )

In [59]:
p.dims

('lon', 'lat', 'Zlayers', 'time')

In [ ]:
depth_tile = np.tile(depth_tile, (len(U.time), 1,len(U.lat),len(U.lon)))

In [68]:
lat_tile = lat_mesh[:,None,None]
lat_tile = np.tile(lat_mesh, (1, len(S.Zlayers), len(S.time)))
lon_tile = np.tile(lon_mesh, (1, len(S.Zlayers), len(S.time)))

In [85]:
lat_tile = np.expand_dims(lat_mesh, (2,3))
lat_tile = np.tile(lat_tile,(1,1, len(S.Zlayers), len(S.time)))
lon_tile = np.expand_dims(lon_mesh,(2,3))
lon_tile = np.tile(lon_tile,(1,1, len(S.Zlayers), len(S.time)))

In [88]:
lat_tile.shape

(980, 786, 10, 24)

In [89]:
p.shape

(980, 786, 10, 24)

In [84]:
np.sum(lat_tile[:,:,0,0] - lat_tile[:,:,1,20])

0.0

In [39]:
p.shape

(980, 786, 10, 24)

In [58]:
p.dtype

dtype('float64')

In [55]:
import gsw_xarray as gswx

In [57]:
gswx.SA_from_SP(S[:,:,0,0], p[:,:,0,0], lon_mesh, lat_mesh)

<xarray.DataArray 'SA' (lon: 980, lat: 786)>
array([[33.32559637, 33.30136981, 33.27413831, ..., 33.03753075,
        33.0455    , 33.0505716 ],
       [33.30760619, 33.29020205, 33.26697202, ..., 33.03849393,
        33.04548197, 33.0511515 ],
       [33.28925189, 33.27904579, 33.26141167, ..., 33.03972159,
        33.04566708, 33.05168157],
       ...,
       [33.17912363, 33.1847817 , 33.18905225, ...,         nan,
                nan,         nan],
       [33.18976802, 33.19359395, 33.19738155, ...,         nan,
                nan,         nan],
       [33.19898273, 33.20374772, 33.21199301, ...,         nan,
                nan,         nan]])
Coordinates:
  * lon      (lon) >f4 -127.6 -127.6 -127.6 -127.6 ... -122.5 -122.5 -122.5
  * lat      (lat) >f4 36.0 36.0 36.01 36.01 36.01 ... 38.98 38.99 38.99 38.99
    Zlayers  float64 88.17
    time     int64 0
Attributes:
    standard_name:  sea_water_absolute_salinity
    units:          g/kg

In [93]:
S

<xarray.DataArray 'Salt' (lon: 980, lat: 786, Zlayers: 10, time: 24)>
array([[[[33.16801834, 33.11510086, 33.11051559, ..., 33.11781311,
          33.12119293, 33.16854095],
         [33.35055161, 33.26220322, 33.24065781, ..., 33.21652603,
          33.23684692, 33.32228851],
         [33.56190109, 33.45243835, 33.41144562, ..., 33.43720245,
          33.4625473 , 33.54219437],
         ...,
         [33.99084091, 33.99086761, 33.9908638 , ..., 33.99052048,
          33.99020386, 33.98989487],
         [33.99085617, 33.99087906, 33.99087524, ..., 33.99053955,
          33.99022675, 33.98992157],
         [33.9908905 , 33.99094009, 33.99093628, ..., 33.99154282,
          33.9913063 , 33.9911232 ]],

        [[33.14390564, 33.12353134, 33.11778259, ..., 33.11929703,
          33.117939  , 33.15394974],
         [33.32202148, 33.26909256, 33.24514771, ..., 33.22018051,
          33.21661758, 33.28952026],
         [33.53947449, 33.4589119 , 33.40802383, ..., 33.44936371,
          33.43308258, 33.50445938],
...
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * lon      (lon) >f4 -127.6 -127.6 -127.6 -127.6 ... -122.5 -122.5 -122.5
  * lat      (lat) >f4 36.0 36.0 36.01 36.01 36.01 ... 38.98 38.99 38.99 38.99
  * Zlayers  (Zlayers) float64 88.17 99.37 111.6 124.9 ... 189.9 209.2 229.9
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23

In [91]:
SAxr_test = gswx.SA_from_SP(S, p, lon_tile, lat_tile)

In [96]:
np.sum(SAxr_test.isnull()) - np.sum(S.isnull())

<xarray.DataArray ()>
array(0)

In [6]:
depth_tile = depth_list[None,:, None,None]
depth_tile = np.tile(depth_tile, (len(U.time), 1,len(U.lat),len(U.lon)))
depth_tile_x = xr.DataArray(np.swapaxes(np.swapaxes(depth_tile, 0,3), 1,2), dims=["lon", "lat","Zlayers", "time"],  coords = { "lon":U.lon, "lat":U.lat, "Zlayers": depth[layer_min:layer_max],  "time":U.time }, name="Depths")

In [ ]:
def calculate_p_joe_xarray(depthsx):
    g = 9.81; rho0 = 1035;
    p = depthsx*g*rho0 / 10000 + 10
    return p.rename('Pressures');

#KEEP IN MIND, JOE'S CODE IS USING IN-SITU TEMPERATURE AND COUPLED SIMULATION SAVE POTENTIAL TEMPERATURE
#all inputs should be xarrays here
def compute_rho_mitgcm_coupledsim_xarray(T, S, depths ):
    # get dims
    import gsw-xarray as gswx
    (nx,ny,nz,nt) = T.shape;
    p = calculate_p_joe_xarray(thk, layer_min, layer_max) 
    
    
    lat_mesh, lon_mesh = np.meshgrid(S.lat, S.lon )
    lat_tile = np.expand_dims(lat_mesh, (2,3))
    lat_tile = np.tile(lat_tile,(1,1, nz, nt))
    lon_tile = np.expand_dims(lon_mesh,(2,3))
    lon_tile = np.tile(lon_tile,(1,1, nz, nt))
    
    SA = gswx.SA_from_SP(S, p, lon_tile, lat_tile)
    CT = gswx.CT_from_pt(SA, T)
    rho = gswx.rho(SA, CT, p)

#the depths here just have to shifted one layer deeper
#do we need a second one of these? If we just need to make the shift, why bother?
def compute_pot_rho_down_mitgcm_coupledsim_xarray(T, S, depths):
        # get dims
        (nx,ny,nz,nt) = self.T.shape;
                
        rho0 = 1035; g = 9.81;
        p = (np.cumsum(self.thknss,axis=2)-0.5*self.thknss)*g*rho0/10000 + 10;
        SA = np.zeros([nx,ny,nz,nt]); CT = np.zeros([nx,ny,nz,nt]); pot_rho = np.zeros([nx,ny,nz,nt]);
        for k in range(0,nz-1):
            for t in range(0,nt):
                SA[:,:,k,t] = gsw.SA_from_SP(self.S[:,:,k,t],p[:,:,k+1,0],np.squeeze(self.pgrid[:,:,0]),np.squeeze(self.pgrid[:,:,1]));
                CT[:,:,k,t] = gsw.CT_from_t(SA[:,:,k,t],self.T[:,:,k,t],p[:,:,k+1,0]);
                pot_rho[:,:,k,t] = gsw.rho(SA[:,:,k,t],CT[:,:,k,t],p[:,:,k+1,0]);

        return pot_rho;